In [1]:
from deep_orderbook.learn.test_learn import train_and_predict
from deep_orderbook.visu import Visualizer
from deep_orderbook.config import TrainConfig, ReplayConfig, ShaperConfig
from deep_orderbook.strategy import Strategy
train_config = TrainConfig(
    num_workers=5,
    batch_size=64,
    data_queue_size=1024,
    num_levels=8,
    learning_rate=0.001,
    epochs=100, save_checkpoint_batches=10, save_checkpoint_mins=1.0
)
replay_config = ReplayConfig(
    markets=["ETH-USD"],#, "BTC-USD", "ETH-BTC"],
    date_regexp='2024-1[0,1]-0*',
    data_dir='/media/photoDS216/crypto/',
    every="1000ms",
)
shaper_config = ShaperConfig(
    only_full_arrays=True,
    view_bips = 20,
    num_side_lvl = 8,
    look_ahead = 32,
    look_ahead_side_bips = 10,
    look_ahead_side_width = 4,
    rolling_window_size = 1024,
    window_stride = 8,
    randomize=True,
)
test_config = replay_config.but(
    date_regexp='2024-11-1*',
    randomize=False
)

In [ ]:
vis = Visualizer()
strategy = Strategy(threshold=0.3)
async def online_learning(train_config, replay_config, shaper_config):
    async for books_arr, t2l, pxar, pred_t2l, train_loss, test_loss in train_and_predict(
        resume_from_checkpoint=True,
        config=train_config, replay_config=replay_config, shaper_config=shaper_config, test_config=test_config
    ):
        gt_pnl, pos, gt_up_prox, gt_down_prox = strategy.compute_pnl(pxar, t2l)
        pred_pnl, pred_pos, pred_up_prox, pred_down_prox = strategy.compute_pnl(pxar, pred_t2l)

        vis.add_loss(train_loss, test_loss)
        vis.update(
            books_z_data=books_arr, level_reach_z_data=t2l, bidask=pxar, 
            pred_t2l=pred_t2l, gt_pnl=gt_pnl, pred_pnl=pred_pnl, 
            positions=pos, pred_positions=pred_pos,
            up_proximity=gt_up_prox, down_proximity=gt_down_prox,
            pred_up_proximity=pred_up_prox, pred_down_proximity=pred_down_prox
        )
await online_learning(train_config=train_config, replay_config=replay_config, shaper_config=shaper_config)